<a href="https://colab.research.google.com/github/shilpa2503/Machine-Learning/blob/main/Differentially_Private_MLP_(Opacus).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import copy
import numpy as np
import torch
import os
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import math

class MLP(nn.Module):
    def __init__(self, dim_in, dim_hidden, dim_out):
        super(MLP, self).__init__()
        self.layer_input = nn.Linear(dim_in, dim_hidden)
        self.relu = nn.ReLU()
        # self.dropout = nn.Dropout()
        self.layer_hidden = nn.Linear(dim_hidden, dim_out)

    def forward(self, x):
        x = self.layer_input(x)
        # x = self.dropout(x)
        x = self.relu(x)
        x = self.layer_hidden(x)
        return x


In [ ]:
def get_dataset():
  df = pd.read_csv('/content/drive/MyDrive/wine.csv').dropna()
  mask = (df.type == 'white')
  df.loc[mask, 'type'] = 0
  mask2 = (df.type == 'red')
  df.loc[mask2, 'type'] = 1

  columns = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
              'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
  df_x = df.drop('type', axis=1)
  df_x = df_x[columns].astype(float)
  df_y = df['type']
  df_y = df_y.astype(int)
  X = df_x.to_numpy()
  Y = df_y.to_numpy()
  x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

  train_dataset = DataLoader(
      TensorDataset(torch.from_numpy(x_train).float(), torch.from_numpy(y_train).long()))

  test_dataset = DataLoader(
      TensorDataset(torch.from_numpy(x_val).float(), torch.from_numpy(y_val).long()))
  return train_dataset, test_dataset

In [ ]:
dataset_train, dataset_test = get_dataset()

In [ ]:
num_classes = 2
len_in = 1
dataset_train = dataset_train.dataset
dataset_test = dataset_test.dataset
data_size = dataset_train[0][0].shape
for x in data_size:
    len_in *= x
net = MLP(dim_in=len_in, dim_hidden=200, dim_out=num_classes)

In [ ]:
def train(net):
  loss_func = nn.CrossEntropyLoss()
  batch_size = 64
  ldr_train = DataLoader((dataset_train), batch_size=batch_size, shuffle=True)

  net.train()
    # train and update

  optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

  epoch_loss = []
  for iter in range(100):
      batch_loss = []
      for batch_idx, (images, labels) in enumerate(ldr_train):
          net.zero_grad()
          log_probs = net(images)

          loss = loss_func(log_probs, labels)

          loss.backward()
          optimizer.step()

          batch_loss.append(loss.item())

      epoch_loss.append(sum(batch_loss) / len(batch_loss))

  return net.state_dict(), sum(epoch_loss) / len(epoch_loss)

In [ ]:
w, loss = train(net)

In [ ]:
net.load_state_dict(w)

<All keys matched successfully>

In [ ]:
import torch.nn.functional as F
from torch.utils.data import DataLoader
def test_img(net, datatest):
    net.eval()
    # testing
    test_loss = 0
    correct = 0
    data_loader = DataLoader(datatest, batch_size=64)
    l = len(data_loader)
    for idx, (data, target) in enumerate(data_loader):
        log_probs = net(data)
        # sum up batch loss
        test_loss += F.cross_entropy(log_probs, target, reduction='sum').item()
        # get the index of the max log-probability
        y_pred = log_probs.data.max(1, keepdim=True)[1]
        correct += y_pred.eq(target.data.view_as(y_pred)).long().cpu().sum()

    test_loss /= len(data_loader.dataset)
    accuracy = 100.00 * correct / len(data_loader.dataset)

    return accuracy, test_loss

In [ ]:
acc_train, loss_train_ = test_img(net, dataset_train)
acc_test, loss_test = test_img(net, dataset_test)

In [ ]:
print("Training accuracy: {:.2f}".format(acc_train))
print("Testing accuracy: {:.2f}".format(acc_test))

Training accuracy: 94.67
Testing accuracy: 94.90
